# Azure Inteligence
Azure Document Intelligence allows extract information from documents.
Below is example of extracting information from Receipts.
Create resource as described here https://microsoftlearning.github.io/mslearn-ai-fundamentals/Instructions/Labs/10-document-intelligence.html then use API as below.

In [ ]:
%pip install azure-ai-formrecognizer

In [1]:
import os, requests, uuid, json
from dotenv import load_dotenv

# endpoint and key are stored in .env file. Load it.
load_dotenv()
print(os.getenv("AZURE_INTELIGENCE_ENDPOINT"))

https://australiaeast.api.cognitive.microsoft.com/


In [6]:
from azure.core.credentials import AzureKeyCredential
from azure.ai.formrecognizer import DocumentAnalysisClient

endpoint = os.getenv("AZURE_INTELIGENCE_ENDPOINT")
key = os.getenv("AZURE_INTELIGENCE_KEY")

# loading sample document here
# either use url
# url = "https://raw.githubusercontent.com/Azure/azure-sdk-for-python/main/sdk/formrecognizer/azure-ai-formrecognizer/tests/sample_forms/receipt/contoso-receipt.png"

# or local file
file_path = "data/receipt.jpg"
with open(file_path, "rb") as f:
    file_content = f.read()

document_analysis_client = DocumentAnalysisClient(
    endpoint=endpoint, credential=AzureKeyCredential(key)
)
# in case of url usage
# poller = document_analysis_client.begin_analyze_document_from_url("prebuilt-receipt", url)

#in case of local file usage
poller = document_analysis_client.begin_analyze_document("prebuilt-receipt", file_content)

receipts = poller.result()

for idx, receipt in enumerate(receipts.documents):
    print("--------Recognizing receipt #{}--------".format(idx + 1))
    receipt_type = receipt.doc_type
    if receipt_type:
        print(
            "Receipt Type: {}".format(receipt_type)
        )
    merchant_name = receipt.fields.get("MerchantName")
    if merchant_name:
        print(
            "Merchant Name: {} has confidence: {}".format(
                merchant_name.value, merchant_name.confidence
            )
        )
    transaction_date = receipt.fields.get("TransactionDate")
    if transaction_date:
        print(
            "Transaction Date: {} has confidence: {}".format(
                transaction_date.value, transaction_date.confidence
            )
        )
    if receipt.fields.get("Items"):
        print("Receipt items:")
        for idx, item in enumerate(receipt.fields.get("Items").value):
            print("...Item #{}".format(idx + 1))
            item_description = item.value.get("Description")
            if item_description:
                print(
                    "......Item Description: {} has confidence: {}".format(
                        item_description.value, item_description.confidence
                    )
                )
            item_quantity = item.value.get("Quantity")
            if item_quantity:
                print(
                    "......Item Quantity: {} has confidence: {}".format(
                        item_quantity.value, item_quantity.confidence
                    )
                )
            item_price = item.value.get("Price")
            if item_price:
                print(
                    "......Individual Item Price: {} has confidence: {}".format(
                        item_price.value, item_price.confidence
                    )
                )
            item_total_price = item.value.get("TotalPrice")
            if item_total_price:
                print(
                    "......Total Item Price: {} has confidence: {}".format(
                        item_total_price.value, item_total_price.confidence
                    )
                )
    subtotal = receipt.fields.get("Subtotal")
    if subtotal:
        print(
            "Subtotal: {} has confidence: {}".format(
                subtotal.value, subtotal.confidence
            )
        )
    tax = receipt.fields.get("TotalTax")
    if tax:
        print("Tax: {} has confidence: {}".format(tax.value, tax.confidence))
    tip = receipt.fields.get("Tip")
    if tip:
        print("Tip: {} has confidence: {}".format(tip.value, tip.confidence))
    total = receipt.fields.get("Total")
    if total:
        print("Total: {} has confidence: {}".format(total.value, total.confidence))
    print("--------------------------------------")


--------Recognizing receipt #1--------
Receipt Type: receipt.retailMeal
Merchant Name: Northwind Traders has confidence: 0.994
Transaction Date: 2020-02-17 has confidence: 0.995
Receipt items:
...Item #1
......Item Description: Apple has confidence: 0.995
......Item Quantity: 1.0 has confidence: 0.995
......Total Item Price: 0.9 has confidence: 0.995
...Item #2
......Item Description: Orange has confidence: 0.995
......Item Quantity: 1.0 has confidence: 0.995
......Total Item Price: 0.8 has confidence: 0.995
Subtotal: 1.7 has confidence: 0.995
Tax: 0.17 has confidence: 0.995
Total: 1.87 has confidence: 0.991
--------------------------------------
